# Translation (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install numpy==1.26.4 datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs
!pip install -q kagglehub transformers torch accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 118.0 MB/s eta 0:00:00
^C


You will need to setup git, adapt your email and name in the following cell.

In [1]:
!git config --global user.email "joao.paulo.migliatti@gmail.com"
!git config --global user.name "psicanalise10"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from google.colab import files
uploaded = files.upload()  # Selecione o kaggle.json baixado do site

Saving kaggle.json to kaggle.json


In [4]:
import kagglehub

# 🔽 Baixar dataset (isso retorna o caminho completo)
path = kagglehub.dataset_download("djonafegnem/europarl-parallel-corpus-19962011")

# 🔍 Ver arquivos disponíveis no diretório baixado
import os
print("Arquivos disponíveis:", os.listdir(path))


100%|██████████| 2.37G/2.37G [01:01<00:00, 41.3MB/s]

Extracting files...


Arquivos disponíveis: ['english_latvia.csv', 'english_lithunian.csv', 'english_finnish.csv', 'english_greek.csv', 'english_estonian.csv', 'english_romanian.csv', 'english_czech.csv', 'english_slovene.csv', 'english_slovak.csv', 'english_polish.csv', 'english_german.csv', 'english_spanish.csv', 'english_italian.csv', 'english_bulgarian.csv', 'english_portuguese.csv', 'english_french.csv', 'english_dutch.csv', 'english_hungarian.csv', 'english_swedish.csv']


In [5]:
import pandas as pd

df = pd.read_csv(os.path.join(path, "english_portuguese.csv"))
df = df.dropna().reset_index(drop=True)
df = df.rename(columns={"English": "en", "Portuguese": "pt"})


In [6]:
import os
import pandas as pd
from datasets import Dataset

# Carrega apenas os primeiros 5.000 exemplos
df = pd.read_csv(os.path.join(path, "english_portuguese.csv")).dropna().reset_index(drop=True)
df = df.rename(columns={"English": "en", "Portuguese": "pt"})
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df = df[:100000]


raw_datasets = Dataset.from_pandas(df[["en", "pt"]])
raw_datasets = raw_datasets.train_test_split(test_size=0.2)


In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['en', 'pt'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['en', 'pt'],
        num_rows: 20000
    })
})

In [8]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['en', 'pt'],
        num_rows: 72000
    })
    test: Dataset({
        features: ['en', 'pt'],
        num_rows: 8000
    })
})

In [9]:
split_datasets["validation"] = split_datasets.pop("test")

In [10]:
from datasets import DatasetDict

def format_translation(example):
    return {"translation": {"en": example["en"], "pt": example["pt"]}}

formatted_train = split_datasets["train"].map(format_translation)

formatted_validation = split_datasets["validation"].map(format_translation)

split_datasets = DatasetDict({
    "train": formatted_train,
    "validation": formatted_validation
})


Map:   0%|          | 0/72000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [11]:
split_datasets["train"][1]["translation"]

{'en': 'Mr Theonas, a debate will take place tomorrow in the presence of the Council and the Commission on the situation in Kosovo and all these problems.',
 'pt': 'Senhor Deputado Theonas, amanhã será realizado um debate, com o Conselho e com a Comissão, sobre a situação no Kosovo e sobre todos esses problemas.'}

In [12]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-mul"
translator = pipeline("translation", model=model_checkpoint)
text = "This is a test sentence."

text_with_prefix = ">>por<< " + text
translator(text_with_prefix)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


[{'translation_text': 'Esta é uma frase de ensaio.'}]

In [13]:
split_datasets["train"][172]["translation"]

{'en': 'If you are elected, Mr\xa0Barroso, how many women will there be in the Commission?',
 'pt': 'Se for eleito, Senhor Barroso, quantas mulheres haverá na Comissão?'}

In [14]:
text = "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
text_with_prefix = ">>por<< " + text
translator(text_with_prefix)

[{'translation_text': 'Não foi posível importar% 1 usando o plugin importador OFX. Este arquivo não é o formato correto.'}]

In [15]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-tc-bible-big-mul-mul"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [16]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
pt_sentence = split_datasets["train"][1]["translation"]["pt"]

inputs = tokenizer(en_sentence, text_target=pt_sentence)
inputs

{'input_ids': [33777, 35534, 14493, 15831, 50, 36415, 40245, 58221, 55333, 51093, 56076, 44550, 55740, 51622, 49923, 55740, 31287, 37059, 55740, 31227, 50092, 55740, 54163, 44550, 33190, 37059, 36886, 55748, 51782, 84, 524], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [35049, 31406, 15154, 7673, 35534, 14493, 15831, 50, 36976, 53908, 52671, 56814, 40245, 50, 39364, 49787, 31254, 41208, 39364, 36415, 31221, 50, 54318, 36415, 54165, 49360, 33190, 41208, 54318, 56035, 41817, 51777, 84, 524]}

In [17]:
wrong_targets = tokenizer(pt_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Senhor', '▁De', 'put', 'ado', '▁The', 'ona', 's', ',', '▁amanhã', '▁será', '▁realizado', '▁um', '▁debate', ',', '▁com', '▁o', '▁Conselho', '▁e', '▁com', '▁a', '▁Comissão', ',', '▁sobre', '▁a', '▁situação', '▁no', '▁Kosovo', '▁e', '▁sobre', '▁todos', '▁esses', '▁problemas', '.', '</s>']
['▁Senhor', '▁De', 'put', 'ado', '▁The', 'ona', 's', ',', '▁amanhã', '▁será', '▁realizado', '▁um', '▁debate', ',', '▁com', '▁o', '▁Conselho', '▁e', '▁com', '▁a', '▁Comissão', ',', '▁sobre', '▁a', '▁situação', '▁no', '▁Kosovo', '▁e', '▁sobre', '▁todos', '▁esses', '▁problemas', '.', '</s>']


In [18]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["pt"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [19]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/72000 [00:00<?, ? examples/s]

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [20]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/991M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

In [21]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [22]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

KeysView({'input_ids': tensor([[33777, 35534, 14493, 15831,    50, 36415, 40245, 58221, 55333, 51093,
         56076, 44550, 55740, 51622, 49923, 55740, 31287, 37059, 55740, 31227,
         50092, 55740, 54163, 44550, 33190, 37059, 36886, 55748, 51782,    84,
           524],
        [35534, 50414, 40793,    50, 44058,    50, 39682, 47481, 43285, 51947,
         15831,    84,   524, 69666, 69666, 69666, 69666, 69666, 69666, 69666,
         69666, 69666, 69666, 69666, 69666, 69666, 69666, 69666, 69666, 69666,
         69666]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0]]), 'labels': tensor([[35049, 31406, 15154,  7673, 35534, 14493, 15831,    50, 36976, 53908,
         52671, 56814, 40245,    50, 39364, 49787, 31254, 41208, 39364, 36415,
         31221,    50, 54318, 36415, 54165, 49360, 3319

In [23]:
batch["labels"]

tensor([[35049, 31406, 15154,  7673, 35534, 14493, 15831,    50, 36976, 53908,
         52671, 56814, 40245,    50, 39364, 49787, 31254, 41208, 39364, 36415,
         31221,    50, 54318, 36415, 54165, 49360, 33190, 41208, 54318, 56035,
         41817, 51777,    84,   524],
        [34012, 41624,    50, 49787, 50401, 14617, 39735, 49531, 15831,  8067,
         51950, 15831, 51362, 15831,    84,   524,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100]])

In [24]:
batch["decoder_input_ids"]

tensor([[69666, 35049, 31406, 15154,  7673, 35534, 14493, 15831,    50, 36976,
         53908, 52671, 56814, 40245,    50, 39364, 49787, 31254, 41208, 39364,
         36415, 31221,    50, 54318, 36415, 54165, 49360, 33190, 41208, 54318,
         56035, 41817, 51777,    84],
        [69666, 34012, 41624,    50, 49787, 50401, 14617, 39735, 49531, 15831,
          8067, 51950, 15831, 51362, 15831,    84,   524, 69666, 69666, 69666,
         69666, 69666, 69666, 69666, 69666, 69666, 69666, 69666, 69666, 69666,
         69666, 69666, 69666, 69666]])

In [25]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[35049, 31406, 15154, 7673, 35534, 14493, 15831, 50, 36976, 53908, 52671, 56814, 40245, 50, 39364, 49787, 31254, 41208, 39364, 36415, 31221, 50, 54318, 36415, 54165, 49360, 33190, 41208, 54318, 56035, 41817, 51777, 84, 524]
[34012, 41624, 50, 49787, 50401, 14617, 39735, 49531, 15831, 8067, 51950, 15831, 51362, 15831, 84, 524]


In [26]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.0 MB/s eta 0:00:00


In [27]:
import evaluate

metric = evaluate.load("sacrebleu")

In [28]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [29]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [30]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

In [31]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [32]:
from huggingface_hub import notebook_login

notebook_login()

In [33]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"joao-finetuned-kde4-en-to-pt",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [35]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-35-543751272.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [36]:
trainer.evaluate(max_length=max_length)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: joao-paulo-migliatti (joao-paulo-migliatti-ufscar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'eval_loss': 10.091519355773926,
 'eval_model_preparation_time': 0.0031,
 'eval_bleu': 0.008823081057684541,
 'eval_runtime': 1008.377,
 'eval_samples_per_second': 7.934,
 'eval_steps_per_second': 0.124}

In [37]:
trainer.train()

Step,Training Loss
500,1.575800
1000,1.354800
1500,1.319700
2000,1.296500
2500,1.230200
3000,1.185400
3500,1.179000
4000,1.171300
4500,1.162700
5000,1.092500


TrainOutput(global_step=6750, training_loss=1.2146649034288195, metrics={'train_runtime': 3355.9278, 'train_samples_per_second': 64.364, 'train_steps_per_second': 2.011, 'total_flos': 2.030045522558976e+16, 'train_loss': 1.2146649034288195, 'epoch': 3.0})

In [38]:
trainer.evaluate(max_length=max_length)

{'eval_loss': 1.1366267204284668,
 'eval_model_preparation_time': 0.0031,
 'eval_bleu': 36.20924445535818,
 'eval_runtime': 915.3069,
 'eval_samples_per_second': 8.74,
 'eval_steps_per_second': 0.137,
 'epoch': 3.0}

In [39]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

events.out.tfevents.1751912177.8c7882597e89.566.1:   0%|          | 0.00/473 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jmigliatti/joao-finetuned-kde4-en-to-pt/commit/592c195c981d11b088b74bac49aff847946d691e', commit_message='Training complete', commit_description='', oid='592c195c981d11b088b74bac49aff847946d691e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Jmigliatti/joao-finetuned-kde4-en-to-pt', endpoint='https://huggingface.co', repo_type='model', repo_id='Jmigliatti/joao-finetuned-kde4-en-to-pt'), pr_revision=None, pr_num=None)

In [40]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [41]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [42]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [43]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [44]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [45]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "joao-finetuned-kde4-en-to-pt"
repo_name = get_full_repo_name(model_name)
repo_name

'Jmigliatti/joao-finetuned-kde4-en-to-pt'

In [46]:
output_dir = "joao-finetuned-kde4-en-to-pt-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/Jmigliatti/joao-finetuned-kde4-en-to-pt into local empty directory.


Download file model.safetensors:   0%|          | 8.74k/945M [00:00<?, ?B/s]

Download file runs/Jul07_15-50-03_8c7882597e89/events.out.tfevents.1751904436.8c7882597e89.566.0: 100%|#######…

Download file runs/Jul01_12-32-18_185f8304624a/events.out.tfevents.1751373194.185f8304624a.6454.0: 100%|######…

Download file runs/Jul06_18-25-06_c51e9d022dbe/events.out.tfevents.1751827814.c51e9d022dbe.1380.0: 100%|######…

...

Download file runs/Jul01_13-12-52_7125e73f2dc8/events.out.tfevents.1751375930.7125e73f2dc8.2970.1: 100%|######…

Clean file runs/Jul01_13-12-52_7125e73f2dc8/events.out.tfevents.1751375930.7125e73f2dc8.2970.1: 100%|#########…

Clean file source.spm:   0%|          | 1.00k/1.25M [00:00<?, ?B/s]

Clean file target.spm:   0%|          | 1.00k/1.25M [00:00<?, ?B/s]

Clean file model.safetensors:   0%|          | 1.00k/945M [00:00<?, ?B/s]

In [47]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/27000 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:741: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "Jmigliatti/joao-finetuned-kde4-en-to-pt"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)